# kaggle Titanic competition

In [327]:
# import required pakages
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

Load the training and testing data.

In [328]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
gender_data = pd.read_csv("data/gender_submission.csv")

### Get insights into the training data
Use `train_data.head(10)` for a tabular overview of the training data.

In [329]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Use `train_data.info()` to get further insight into the training data.

In [330]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


`train_data.info()` reveals some missing data within the columns `Age`, `Cabin`and `Embarked`.
<br><br>
There are three possibilities to **handle missing values**:
* delete the entries with the missing values
* delete the hole attribute with the missing values
* fill the missing values with the median value of that attribute
<br>

In this case we will use all three different methods to handle the missing values:
* The `Cabin`attribute will be deleted.
* The two entries with the missing `Embarked` value will be deleted.
* The missing values within the `Age` attribute will be filled by the median of the other values of that attribute.
<br>

Furthermore `train_data.info()` reveals that the attributes have three data types: 
* `int64` -> which is okay for further processing.
* `float64` -> which is okay for further processing.
* `object` -> these attributes have to be adopted for further processing.
<br>

First let's have a look at these object attributes and their importance for the upcoming machine-learning process:
* `Name` is an individual value with no valuable information and therefore not important for further processing.
* `Sex` is important for further processing. We will use one-hot-encoding to make this attribute processable.
* `Ticket` is an individual number with no valuable information and therefore not important for further processing.
* `Cabin` will be deleted because there are just 204 out of 981 values. 
* `Embarked` can be interesting for further processing. We will use one-hot-encoding to make this attribute processable.

Use `train_data.describe()` to get an insight of the numerical attributes especially their variance.

In [331]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


There are two attributes that should undergo additional preprocessing:
* `Age` with a standard deviation of 14.52 and a max / min difference of 79.58.
* `Fare` with a standard deviation of 49.69 and a max / min difference of about 512.
<br>

Both attributes will undergo a standardization according the following formula:

$$ x_j^{(i)} = \frac{x_j^{(i)} - \mu_j}{s_j} \\ $$

Here $x_j^{(i)}$ is the value of attribute $j$ of the $i's$ example. $\mu_j$ is the mean of attribute $j$ and $s_j$ is the standard deviation of attribute $j$.

check the correlation between the numerical values:

In [332]:
train_data.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [333]:
train_data.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


### Prepare the training data

 Delete the two entries with the missing *Embarked* attribute

In [334]:
#train_data = train_data.dropna(subset=["Embarked"])
train_data.dropna(subset=["Embarked"], inplace=True)
train_data.reset_index(inplace=True)
train_data.tail()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
884,886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
885,887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
886,888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
887,889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
888,890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


Drop the label attribute `Survived` from the training data set and move it into the label set `y_train`.

In [335]:
y_train = train_data['Survived'].copy()
X_train = train_data.drop('Survived', axis=1)
X_train.head()

,index,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Remove the attributes `index`, `PassengerId`, `Name`, `Ticket` and `Cabin` from the training set.

In [336]:
X_train = X_train.drop(['index','PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [337]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 7 columns):
Pclass      889 non-null int64
Sex         889 non-null object
Age         712 non-null float64
SibSp       889 non-null int64
Parch       889 non-null int64
Fare        889 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 48.7+ KB


In [338]:
sample_incomplete_rows = X_train[X_train.isnull().any(axis=1)].head(10)
sample_incomplete_rows

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,3,male,NaN,0,0,8.4583,Q
17,2,male,NaN,0,0,13.0000,S
19,3,female,NaN,0,0,7.2250,C
26,3,male,NaN,0,0,7.2250,C
28,3,female,NaN,0,0,7.8792,Q
29,3,male,NaN,0,0,7.8958,S
31,1,female,NaN,1,0,146.5208,C
32,3,female,NaN,0,0,7.7500,Q
36,3,male,NaN,0,0,7.2292,C
42,3,male,NaN,0,0,7.8958,C


#### Replace missing values (by hand)

There is also the possibility to replace the missing values with the sklearn.impute.SimpleImputer class.
Use [`sklearn.impute.SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) class for replacing missing values.
<br>
We replace the missing values here by hand because we want to distinguish between the classes and within the classes between the gender. 

Calculate the median values depending on the class and the gender.

In [339]:
medianAges = np.zeros((2,3))

for i in range(0, 2):
    for j in range(0, 3):
        if(i == 0):
            medianAges[i, j] = X_train.loc[(X_train.Pclass == j+1) & (X_train.Sex == 'female')]['Age'].median()
        else:
            medianAges[i, j] = X_train.loc[(X_train.Pclass == j+1) & (X_train.Sex == 'male')]['Age'].median()

medianAges

array([[35. , 28. , 21.5],
       [40. , 30. , 25. ]])

In [340]:
medianAgeFemale_class1 = X_train.loc[(X_train.Pclass == 1) & (X_train.Sex == 'female')]['Age'].median()
medianAgeFemale_class1

35.0

In [341]:
medianAgeMale_class1 = X_train.loc[(X_train.Pclass == 1) & (X_train.Sex == 'male')]['Age'].median()
medianAgeMale_class1

40.0

In [342]:
medianAgeFemale_class2 = X_train.loc[(X_train.Pclass == 2) & (X_train.Sex == 'female')]['Age'].median()
medianAgeFemale_class2

28.0

In [343]:
medianAgeMale_class2 = X_train.loc[(X_train.Pclass == 2) & (X_train.Sex == 'male')]['Age'].median()
medianAgeMale_class2

30.0

In [344]:
medianAgeFemale_class3 = X_train.loc[(X_train.Pclass == 3) & (X_train.Sex == 'female')]['Age'].median()
medianAgeFemale_class3

21.5

In [345]:
medianAgeMale_class3 = X_train.loc[(X_train.Pclass == 3) & (X_train.Sex == 'male')]['Age'].median()
medianAgeMale_class3

25.0

Add the missing `Age` values to X_train

In [346]:
def addMissingAgeValues(dataset, medianAges):
    for i in range(medianAges.shape[0]):
        for j in range(medianAges.shape[1]):
            if(i == 0):
                X_train.loc[(dataset.Pclass == j+1) & (dataset.Age.isnull()) 
                            & (dataset.Sex == 'female'), 'Age'] = medianAges[i, j]
            else:
                X_train.loc[(dataset.Pclass == j+1) & (dataset.Age.isnull()) 
                            & (dataset.Sex == 'male'), 'Age'] = medianAges[i, j]

In [347]:
addMissingAgeValues(X_train, medianAges)

In [348]:
#X_train.loc[(X_train.Pclass == 1) & (X_train.Age.isnull())
#            & (X_train.Sex == 'female'), 'Age'] = medianAgeFemale_class1
#X_train.loc[(X_train.Pclass == 1) & (X_train.Age.isnull()) 
#            & (X_train.Sex == 'male'), 'Age'] = medianAgeMale_class1
#X_train.loc[(X_train.Pclass == 2) & (X_train.Age.isnull()) 
#            & (X_train.Sex == 'female'), 'Age'] = medianAgeFemale_class2
#X_train.loc[(X_train.Pclass == 2) & (X_train.Age.isnull()) 
#            & (X_train.Sex == 'male'), 'Age'] = medianAgeMale_class2
#X_train.loc[(X_train.Pclass == 3) & (X_train.Age.isnull()) 
#            & (X_train.Sex == 'female'), 'Age'] = medianAgeFemale_class3
#X_train.loc[(X_train.Pclass == 3) & (X_train.Age.isnull()) 
#            & (X_train.Sex == 'male'), 'Age'] = medianAgeMale_class3

Check if the replacement was successful.

In [349]:
X_train.loc[sample_incomplete_rows.index.values]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
5,3,male,25.0,0,0,8.4583,Q
17,2,male,30.0,0,0,13.0000,S
19,3,female,21.5,0,0,7.2250,C
26,3,male,25.0,0,0,7.2250,C
28,3,female,21.5,0,0,7.8792,Q
29,3,male,25.0,0,0,7.8958,S
31,1,female,35.0,1,0,146.5208,C
32,3,female,21.5,0,0,7.7500,Q
36,3,male,25.0,0,0,7.2292,C
42,3,male,25.0,0,0,7.8958,C


In [350]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 7 columns):
Pclass      889 non-null int64
Sex         889 non-null object
Age         889 non-null float64
SibSp       889 non-null int64
Parch       889 non-null int64
Fare        889 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(2)
memory usage: 48.7+ KB


### Preprocessing categorical attributes
Extract the categorical attributes `Sex` and `Embarked` and store them within cat_attributes. 

In [351]:
cat_attributes = X_train[['Sex', 'Embarked']]
cat_attributes.head()

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S


#### One-Hot-Encoder
Use [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to transform the categorical values of the attributes `Sex` and `Embarked` into numerical values. The result is a matrix with $n$ columns representing the $n$ different categorical values. In this case $n = 5$. We do not want a sparse matrix but an array, therefore we use `sparse=False`.

In [352]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(sparse=False)
X_train_1hot = cat_encoder.fit_transform(cat_attributes)
X_train_1hot

array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 1.],
       [0., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0.]])

Get the category names out of the cat_encoder and store them within a list.

In [353]:
categories_1hot = cat_encoder.categories_
categories_1hot_list = []
for i in range(len(categories_1hot)):
    for j in range(len(categories_1hot[i])):
        categories_1hot_list.append(categories_1hot[i][j])
    
categories_1hot_list

['female', 'male', 'C', 'Q', 'S']

Delete the categorical attributes from X-train. They will be replaced by the one-hot encoded values.

In [354]:
X_train = X_train.drop(['Sex', 'Embarked'], axis = 1)
X_train.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


Convert the array of one-hot encoder into a pandas DataFrame.

In [355]:
X_train_1hot = pd.DataFrame(X_train_1hot, columns=categories_1hot_list)
X_train_1hot.head(10)

,female,male,C,Q,S
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,1.0
5,0.0,1.0,0.0,1.0,0.0
6,0.0,1.0,0.0,0.0,1.0
7,0.0,1.0,0.0,0.0,1.0
8,1.0,0.0,0.0,0.0,1.0
9,1.0,0.0,1.0,0.0,0.0


In [356]:
X_train_1hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 5 columns):
female    889 non-null float64
male      889 non-null float64
C         889 non-null float64
Q         889 non-null float64
S         889 non-null float64
dtypes: float64(5)
memory usage: 34.8 KB


Concatenate the one-hot encoded categories within `X_train_1hot` to `X_train` 

In [357]:
X_train = pd.concat([X_train, X_train_1hot], axis=1)
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
2,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
4,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [358]:
classes_cat = X_train['Pclass']
classes_cat = classes_cat.as_matrix()
type(classes_cat)

/home/horst/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


numpy.ndarray

In [359]:
classes_cat = classes_cat.reshape(-1, 1) 
classes_cat = cat_encoder.fit_transform(classes_cat)

In [360]:
classes_cat_1hot = pd.DataFrame(classes_cat, columns=['class1', 'class2', 'class3'])
classes_cat_1hot.head(10)

,class1,class2,class3
0,0.0,0.0,1.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
5,0.0,0.0,1.0
6,1.0,0.0,0.0
7,0.0,0.0,1.0
8,0.0,0.0,1.0
9,0.0,1.0,0.0


In [361]:
X_train.head()

,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
2,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
3,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
4,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [362]:
X_train.drop('Pclass', axis=1, inplace=True)
X_train.head()

,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0
1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0
2,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0
3,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0
4,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0


In [363]:
X_train = pd.concat([X_train, classes_cat_1hot], axis=1)
X_train.head()

,Age,SibSp,Parch,Fare,female,male,C,Q,S,class1,class2,class3
0,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [364]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889 entries, 0 to 888
Data columns (total 12 columns):
Age       889 non-null float64
SibSp     889 non-null int64
Parch     889 non-null int64
Fare      889 non-null float64
female    889 non-null float64
male      889 non-null float64
C         889 non-null float64
Q         889 non-null float64
S         889 non-null float64
class1    889 non-null float64
class2    889 non-null float64
class3    889 non-null float64
dtypes: float64(10), int64(2)
memory usage: 83.4 KB


Use StandardScaler from sklearn to scale the attribute values.
StandardScaler uses the method:
$$ x_j^{(i)} = \frac{x_j^{(i)} - \mu_j}{s_j} \\ $$

Again $x_j^{(i)}$ is the value of attribute $j$ of the $i's$ example. $\mu_j$ is the mean of attribute $j$ and $s_j$ is the standard deviation of attribute $j$.

In [365]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
df = pd.DataFrame(scale.fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
#df

In [371]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#scaler.fit_transform(X_train[['Fare']].values)
scaled_values = scaler.fit_transform(X_train)
scaled_values.shape
#scaled_values = X_train

/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


(889, 12)

**Now the training data has been prepared for machine learning!**

* The data has been loaded.
* The data has been studied. 
* The correlation between the numerical attributed has been performed.
* The "label" have been separated from the training data.
* We have checked for and handled missing (numerical) values within the attributes.
* Categorical attributes have been preprocessed by one-hot encoding.
* Scale the date to fit into the machine learning algorithms

**Results of the preprocessing steps:**
* `X_train`: the prepared training data
* `y_train`: the labeled data for training

## Train and evaluate different classification models
Use [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) for model evaluation.

In [372]:
scaled_values = scaled_values[:, 0:7]

In [373]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

**Logistic Regression model:** [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [374]:
from sklearn.linear_model import LogisticRegression

parameters = {'solver':('newton-cg', 'lbfgs', 'liblinear', 'sag'), 'C':[0.1, 1, 10], 'max_iter':[100, 1000]}
log_reg = LogisticRegression(random_state=0, max_iter=100)
grid_log_reg = GridSearchCV(log_reg, parameters, cv=5, return_train_score=False)
grid_log_reg.fit(scaled_values, y_train)

grid_log_results = grid_log_reg.cv_results_
print('Best estimator:', grid_log_reg.best_estimator_)
print('Best score:', grid_log_reg.best_score_)
#pd.DataFrame(grid_log_results)

Best estimator: LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)
Best score: 0.7885264341957255


**SVC:**[`SVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

In [375]:
from sklearn.svm import SVC, LinearSVC

parameters = {'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 'C':[0.1, 1, 10],}
svc_clas = SVC(gamma='scale')
grid_svc = GridSearchCV(svc_clas, parameters, cv=5, return_train_score=False)
grid_svc.fit(scaled_values, y_train)

grid_svc_results = grid_svc.cv_results_
print('Best estimator:', grid_svc.best_estimator_)
print('Best score:', grid_svc.best_score_)

Best estimator: SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Best score: 0.8233970753655793


**LinearSVC:**[`LinearSVC`](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC)

In [376]:
parameters = {'C':[0.1, 1, 10],}
lsvc_clas = LinearSVC(max_iter=100000)
grid_lsvc = GridSearchCV(lsvc_clas, parameters, cv=5, return_train_score=False)
grid_lsvc.fit(scaled_values, y_train)

grid_lsvc_results = grid_lsvc.cv_results_
print('Best estimator:', grid_lsvc.best_estimator_)
print('Best score:', grid_lsvc.best_score_)

Best estimator: LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
Best score: 0.7784026996625422


**SGDClassifier:**[`SGDClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)

In [141]:
from sklearn.linear_model import SGDClassifier

parameters = {}

sgd_clas = SGDClassifier(max_iter=1000, tol=1e-3)
grid_sgd = GridSearchCV(sgd_clas, parameters, cv=5, return_train_score=False)
grid_sgd.fit(scaled_values, y_train)

grid_sgd_results = grid_sgd.cv_results_
print('Best estimator:', grid_sgd.best_estimator_)
print('Best score:', grid_sgd.best_score_)

Best estimator: SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=0.001,
       validation_fraction=0.1, verbose=0, warm_start=False)
Best score: 0.7750281214848144


**DecisionTreeClassifier:** [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

In [142]:
from sklearn.tree import DecisionTreeClassifier

tree_clas = DecisionTreeClassifier()

parameters = {'min_samples_split':[2, 4, 6, 8, 10, 20, 30, 35, 40, 45], 'min_samples_leaf':[1, 2, 3, 4, 6, 8]}
grid_tree = GridSearchCV(tree_clas, parameters, cv=5, return_train_score=False)
grid_tree.fit(scaled_values, y_train)

grid_tree_results = grid_tree.cv_results_
print('Best estimator:', grid_tree.best_estimator_)
print('Best score:', grid_tree.best_score_)

Best estimator: DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=40,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Best score: 0.8143982002249719


**RandomForestClassifier:**[`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)

In [143]:
from sklearn.ensemble import RandomForestClassifier

forest_clas = RandomForestClassifier(n_estimators=100)

parameters = {'min_samples_split':[2, 4, 6, 8, 10, 20, 30, 35, 40, 45], 'min_samples_leaf':[1, 2, 3, 4, 6, 8]}
grid_forest = GridSearchCV(forest_clas, parameters, cv=5, return_train_score=False)
grid_forest.fit(scaled_values, y_train)

grid_forest_results = grid_forest.cv_results_
print('Best estimator:', grid_forest.best_estimator_)
print('Best score:', grid_forest.best_score_)

Best estimator: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Best score: 0.8233970753655793


**KNeighborsClassifier:**[`KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)

In [144]:
from sklearn.neighbors import KNeighborsClassifier

neighbor_clas = KNeighborsClassifier()

parameters = {'algorithm':('auto', 'ball_tree', 'kd_tree', 'brute')}
grid_neighbor = GridSearchCV(neighbor_clas, parameters, cv=5, return_train_score=False)
grid_neighbor.fit(scaled_values, y_train)

grid_neighbor_results = grid_neighbor.cv_results_
print('Best estimator:', grid_neighbor.best_estimator_)
print('Best score:', grid_neighbor.best_score_)

Best estimator: KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
Best score: 0.7975253093363329


**GaussianNB:**[`GaussianNB`](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)

In [145]:
from sklearn.naive_bayes import GaussianNB

gauss_clas = GaussianNB()

parameters = {}
grid_gauss = GridSearchCV(gauss_clas, parameters, cv=5, return_train_score=False)
grid_gauss.fit(scaled_values, y_train)

grid_gauss_results = grid_gauss.cv_results_
print('Best estimator:', grid_gauss.best_estimator_)
print('Best score:', grid_gauss.best_score_)

Best estimator: GaussianNB(priors=None, var_smoothing=1e-09)
Best score: 0.7885264341957255


**Perceptron:**[`Perceptron`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html#sklearn.linear_model.Perceptron)

In [146]:
from sklearn.linear_model import Perceptron

perceptron_clas = Perceptron(max_iter=1000, tol=1e-3)

parameters = {'penalty':(None, 'l2', 'l1', 'elasticnet'), 
              'alpha':[1, 0.3, 0.1, 0.03, 0.01, 0.003, 0.001, 0.0003, 0.0001], 
              'max_iter':[100, 300, 1000]}
grid_perceptron = GridSearchCV(perceptron_clas, parameters, cv=5, return_train_score=False)
grid_perceptron.fit(scaled_values, y_train)

grid_perceptron_results = grid_perceptron.cv_results_
print('Best estimator:', grid_perceptron.best_estimator_)
print('Best score:', grid_perceptron.best_score_)

Best estimator: Perceptron(alpha=0.001, class_weight=None, early_stopping=False, eta0=1.0,
      fit_intercept=True, max_iter=100, n_iter=None, n_iter_no_change=5,
      n_jobs=None, penalty='l1', random_state=0, shuffle=True, tol=0.001,
      validation_fraction=0.1, verbose=0, warm_start=False)
Best score: 0.7772778402699663


/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


**MLPClassifier:**[`MLPClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)

In [285]:
from sklearn.neural_network import MLPClassifier

nn_clas = MLPClassifier(hidden_layer_sizes=(200, 100, 50), max_iter=1000, random_state=1)

parameters = {'activation':('identity', 'logistic', 'tanh', 'relu'), 
              'alpha':[1, 0.3, 0.1, 0.03, 0.01, 0.003, 0.001, 0.0003, 0.0001], 
              'solver':('lbfgs', 'sgd', 'adam')}
grid_nn = GridSearchCV(nn_clas, parameters, cv=5, return_train_score=False)
grid_nn.fit(scaled_values, y_train)

grid_nn_results = grid_nn.cv_results_
print('Best estimator:', grid_nn.best_estimator_)
print('Best score:', grid_nn.best_score_)

/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged 

Best estimator: MLPClassifier(activation='relu', alpha=1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 100, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)
Best score: 0.8323959505061868


/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [147]:
from sklearn.neural_network import MLPClassifier

nn_clas = MLPClassifier(hidden_layer_sizes=(200, 100, 50), max_iter=1000, random_state=1)

parameters = {'activation':['relu'], 
              'alpha':[1], 
              'solver':['sgd']}
grid_nn = GridSearchCV(nn_clas, parameters, cv=5, return_train_score=False)
grid_nn.fit(scaled_values, y_train)

grid_nn_results = grid_nn.cv_results_
print('Best estimator:', grid_nn.best_estimator_)
print('Best score:', grid_nn.best_score_)

/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged 

Best estimator: MLPClassifier(activation='relu', alpha=1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 100, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)
Best score: 0.81214848143982


/home/horst/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [274]:
from sklearn.neural_network import MLPClassifier

nn_clas = MLPClassifier(hidden_layer_sizes=(200, 100, 50), alpha=0.0001, max_iter=1000, random_state=1)
nn_clas.fit(scaled_values, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(200, 100, 50), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [275]:
y_pred = nn_clas.predict(scaled_values)
accuracy_score(y_train, y_pred)

0.8852643419572553